In [2]:
import sagemaker
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()
# bucket = session.default_bucket()
bucket = 'asurion-ml-experimentation'
prefix = 'custom_preprocessing'

In [25]:
label_col = 'nbr_mobile_home_policies'
cat_feats = ['zip_agg_customer_subtype', 'zip_agg_customer_main_type']
# INPUT_FEATURES_SIZE = 85
nbr_cols = df.shape[1]

In [4]:
df = pd.read_csv('s3://asurion-ml-experimentation/custom_preprocessing/data/1_full_data.csv')
df.head()

,zip_agg_customer_subtype,zip_agg_number_of_houses,zip_agg_avg_size_household,zip_agg_avg_age,zip_agg_customer_main_type,zip_agg_roman_catholic,zip_agg_protestant,zip_agg_other_religion,zip_agg_no_religion,zip_agg_married,...,nbr_private_accident_ins_policies,nbr_family_accidents_ins_policies,nbr_disability_ins_policies,nbr_fire_policies,nbr_surfboard_policies,nbr_boat_policies,nbr_bicycle_policies,nbr_property_ins_policies,nbr_ss_ins_policies,nbr_mobile_home_policies
0,Lower class large families,1,3,2,Family with grown ups,0,5,1,3,7,...,0,0,0,1,0,0,0,0,0,0
1,Mixed small town dwellers,1,2,2,Family with grown ups,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,0
2,Mixed small town dwellers,1,2,2,Family with grown ups,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,0
3,"Modern, complete families",1,3,3,Average Family,2,3,2,4,5,...,0,0,0,1,0,0,0,0,0,0
4,Large family farms,1,4,2,Farmers,1,4,1,4,7,...,0,0,0,1,0,0,0,0,0,0


In [19]:
pd.set_option('max_rows', 500)

df.isna().sum()

zip_agg_customer_subtype                 0
zip_agg_number_of_houses                 0
zip_agg_avg_size_household               0
zip_agg_avg_age                          0
zip_agg_customer_main_type               0
zip_agg_roman_catholic                   0
zip_agg_protestant                       0
zip_agg_other_religion                   0
zip_agg_no_religion                      0
zip_agg_married                          0
zip_agg_living_together                  0
zip_agg_other_relation                   0
zip_agg_singles                          0
zip_agg_household_without_children       0
zip_agg_household_with_children          0
zip_agg_high_level_education             0
zip_agg_medium_level_education           0
zip_agg_lower_level_education            0
zip_agg_high_status                      0
zip_agg_entrepreneur                     0
zip_agg_farmer                           0
zip_agg_middle_management                0
zip_agg_skilled_labourers                0
zip_agg_uns

In [45]:
for col in df.columns.difference(cat_feats):
    print(col)
    display(df[col].value_counts())
    print('\n')

contri_agricultural_machines_policies


0    9790
4      11
3       8
2       7
6       5
1       1
Name: contri_agricultural_machines_policies, dtype: int64



contri_bicycle_policies


0    9573
1     249
Name: contri_bicycle_policies, dtype: int64



contri_boat_policies


...

5    1953
4    1929
3    1845
2    1468
6     800
0     634
1     478
7     396
9     202
8     117
Name: zip_agg_social_class_c, dtype: int64



zip_agg_social_class_d


0    4376
1    2658
2    1434
3     757
4     378
5     158
6      37
7      22
9       1
8       1
Name: zip_agg_social_class_d, dtype: int64



zip_agg_unskilled_labourers


2    2460
3    1817
1    1685
0    1636
4    1289
5     564
6     216
7     101
9      37
8      17
Name: zip_agg_unskilled_labourers, dtype: int64

Numeric features have already been standardized or bucketed to 0-10.

In [5]:
train, test = train_test_split(df, test_size = .1, random_state=12, stratify=df[label_col])

In [11]:
train_X = train.iloc[:, :nbr_cols-1]
train_y = train.iloc[:, -1]

In [ ]:
%%writefile custom_preprocess.py
import pandas as pd
import numpy as np

import time
import sys
from io import StringIO
import os
import shutil

import argparse
import csv
import joblib
import json

from sklearn.base import BaseEstimator, TransformerMixin
from sklear.compuse import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--label_col', type=str, default='nbr_mobile_home_policies')
    args = parser.parse_args()
    
    cat_feats = ['zip_agg_customer_subtype', 'zip_agg_customer_main_type']
    nbr_cols = df.shape[1]
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))

    raw_data = [ pd.read_csv(file) for file in input_files ]
    concat_data = pd.concat(raw_data)
    
    number_of_columns_x = concat_data.shape[1]
    train_y = concat_data.iloc[:,number_of_columns_x-1]
    train_X = concat_data.iloc[:,:number_of_columns_x-1]
    
    col_transformer = ColumnTransformer([
        transformers=[
            ('encoder', OneHotEncoder(), cat_feats)]],
        remainder='passthrough')
        
    processed_df = col_transformer.fit_transform(train_X, train_y)

    joblib.dump(feature_selection_pipe, os.path.join(args.model_dir, "model.joblib"))

    print("saved model!")

    # print('OneHot feature names:', feature_selection_pipe.named_steps['one_hot']._all_cols)
    # print('SVR feature names:', feature_selection_pipe.named_steps['svr'].get_support())
    # print('f_reg feature names:', feature_selection_pipe.named_steps['f_reg'].get_support())
    # print('mut_info feature names:', feature_selection_pipe.named_steps['mut_info'].get_support())
    
    # feature_names = feature_selection_pipe.named_steps['one_hot']._all_cols
    # feature_names = feature_names[feature_selection_pipe.named_steps['svr'].get_support()]
    # feature_names = feature_names[feature_selection_pipe.named_steps['f_reg'].get_support()]
    # feature_names = feature_names[feature_selection_pipe.named_steps['mut_info'].get_support()]
    # joblib.dump(feature_names, os.path.join(args.model_dir, "selected_feature_names.joblib"))
    
    # print("Selected features are: {}".format(feature_names))
    
def input_fn(input_data, content_type):
    '''Parse input data payload
    
    Accepts csv, parquet, or json file types'''
    
    print('Running input function')
    
    if content_type == 'text/csv':
        df = pd.read_csv(StringIO(input_data))
        return df
    elif content_type == 'application/x-parquet':
        df = pd.read_parquet(input_data)
    elif content_type == 'application/json':
        df = pd.read_json(input_data)
    else:
        raise ValueError("{} not supported by script".format(content_type))
        
def output_fn(prediction, accept):
    '''Format prediction output.
    
    The default accept/content-type between containers for serial inference is JSON.
    We also want to set the ContentType or mimetype as the same value as accept so the next
    container can read the response payload correctly.
    '''
    
    print('Running output function')
    
    if accept == 'application/json':
        instances = []
        for row in prediction.tolist():
            instances.append({'features': row})
            
        json_output = {'instances': instances}
        
        return worker.Response(json.dumps(json_output), mimetype=accept)
    elif accept == 'text/csv':
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException('{} accept type is not supported by this script')
        
def predict_fn(input_data, model):
    '''Preprocess input data
    
    The default predict_fn uses .predict(), but our model is a preprocessor
    so we want to use .transform().
    '''
    
    print('Running predict_function')
    
    print('Input data shape at predict_fn: {}'.format(input_data.shape))
    if input_data.shape[1] == INPUT_FEATURES_SIZE:
        features = model.transform(input_data)
        return features
    elif input_data.shape[1] -- INPUT_FEATURES_SIZE + 1:
        features = model.transform(inputdata.iloc[:, :INPUT_FEATURES_SIZE])
        return np.insert(features, 0, input_data[label_column], axis=1)
    
def model_fn(model_dir):
    '''Deserialize fitted model'''
    
    print('Running model function')
    
    preprocessor = joblib.load(os.path.join(model_dir, 'model.joblib'))
    return processor